# [Research Request - Explore data for PUC 99314.11 leg report #1613](https://github.com/cal-itp/data-analyses/issues/1613)

Discussed with Peter and Cayman about PUC 99314.11, specifically about sub-section (d)(1)

>(d) (1) On or before November 30, 2025, the department shall submit a report to the Legislature on the revenue vehicle hours, ridership, and passenger mile impacts on the services offered by operators to which Sections 99314.6 and 99314.7 do not apply pursuant to subdivision (a).

1. read in SCO data
2. read in warehouse data for annual ntd reporters

Compare both sources, see which agencies join or not join in either list. 

Compare data in both sources, see if any annual totals match

In [2]:
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

## Read in SCO data
Data is speard out among different excel files for different state fiscal years

In [ ]:
url_23_24 = "https://bythenumbers.sco.ca.gov/download/bdqr-pszz/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
url_21_22 = "https://bythenumbers.sco.ca.gov/download/aqpg-as24/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"
url_18_20 = "https://bythenumbers.sco.ca.gov/download/6dj3-r4jw/application%2Fvnd.openxmlformats-officedocument.spreadsheetml.sheet"

all_years = [
    url_23_24,
    url_21_22,
    url_18_20   
]

In [ ]:
# what are the sheet names for each excel file?
for data in all_years:
    sco_info = pd.ExcelFile(data)
    print(sco_info.sheet_names)

In [ ]:
def make_sco_df(url:str, sheet:str) -> pd.DataFrame:
    """
    Function that reads in each excel url at specific sheet name. then snakecases all the columns
    """
    df = pd.read_excel(url, sheet_name = sheet)
    df = to_snakecase(df)
    
    return df

In [ ]:
# make DFs for each excel sheet
ops_18_20 = make_sco_df(url_18_20, "TO_OPERATING_DATA")
ops_21_22 = make_sco_df(url_21_22, "5 TO_OPERATING_DATA")
ops_23_24 = make_sco_df(url_23_24, "5 TO_OPERATING_DATA")

In [ ]:
# examine each sheet was read in successfully
display(
    ops_18_20.shape,
    ops_21_22.shape,
    ops_23_24.shape
)

## SCO data cleaning

In [ ]:
# do all DFs have the same columns?
set(ops_18_20.columns) == set(ops_21_22.columns) == set(ops_23_24.columns) #TRUE!

In [ ]:
# group column names by similar categories 

date_cols =[
    'date_service_began_operations__mm_dd_yyyy__motor_bus',
    'date_service_began_operations__mm_dd_yyyy__heavy_rail',
    'date_service_began_operations__mm_dd_yyyy__light_rail',
    'date_service_began_operations__mm_dd_yyyy__trolley_bus',
    'date_service_began_operations__mm_dd_yyyy__ferry_boat',
    'date_service_began_operations__mm_dd_yyyy__demand_response_vehicles',
    'date_service_began_operations__mm_dd_yyyy__vanpool',
    'date_service_began_operations__mm_dd_yyyy__other_transportation_mode__specify_'
]

vrh_cols = [
    "total_actual_vehicle_revenue_hours_—_annual_demand_response_vehicles_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_ferry_boat_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_heavy_rail_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_light_rail_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_motor_bus_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_other_transportation_mode__specify__actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_trolley_bus_actual_vehicle_revenue_hours_—_annual",
    "total_actual_vehicle_revenue_hours_—_annual_vanpool_actual_vehicle_revenue_hours_—_annual",
]

vrm_cols = [
    "total_actual_vehicle_revenue_miles_demand_response_vehicles",
    "total_actual_vehicle_revenue_miles_ferry_boat",
    "total_actual_vehicle_revenue_miles_heavy_rail",
    "total_actual_vehicle_revenue_miles_light_rail",
    "total_actual_vehicle_revenue_miles_motor_bus",
    "total_actual_vehicle_revenue_miles_other_transportation_mode__specify_",
    "total_actual_vehicle_revenue_miles_trolley_bus",
    "total_actual_vehicle_revenue_miles_vanpool",
]

passenger_cols = [
    "total_passengers_—_annual_demand_response_vehicles",
    "total_passengers_—_annual_ferry_boat",
    "total_passengers_—_annual_heavy_rail",
    "total_passengers_—_annual_light_rail",
    "total_passengers_—_annual_motor_bus",
    "total_passengers_—_annual_other_transportation_mode__specify_",
    "total_passengers_—_annual_trolley_bus",
    "total_passengers_—_annual_vanpool",
]

group_list = ['entity_name', 'fiscal_year', 'entity_id']

In [ ]:
# do all DFs have the same column datatypes?
display(
    ops_18_20.dtypes.equals(ops_21_22.dtypes),
    ops_18_20.dtypes.equals(ops_23_24.dtypes),
    ops_21_22.dtypes.equals(ops_23_24.dtypes),
)

# op_18_20 data types dont match. 


In [ ]:
# columns dont match?
col_cats ={
    "date columns":date_cols,
    "vrh columns":vrh_cols,
    "vrm columns":vrm_cols,
    "passenger columns":passenger_cols
}

for name, cols in col_cats.items():
    print(f"""Check if data types in {name} match:
    FY 18-20 vs 21-22: {ops_18_20[cols].dtypes.equals(ops_21_22[cols].dtypes)},
    FY 18-20 vs 23-24: {ops_18_20[cols].dtypes.equals(ops_23_24[cols].dtypes)},
    """)
    

In [ ]:
all_ops = [
    ops_18_20,
    ops_21_22,
    ops_23_24
]

# what do the date column data types look like in each df?
for df in all_ops:
    display(
        df[date_cols].info()
    )
    
# ops_18_20.date_service_began_operations__mm_dd_yyyy__demand_response_vehicles is an object, everything else is datetime64[ns]

In [ ]:
# attempt to change data type from object to datetime64[ns]?
ops_18_20[
    "date_service_began_operations__mm_dd_yyyy__demand_response_vehicles"] = pd.to_datetime(ops_18_20[
    "date_service_began_operations__mm_dd_yyyy__demand_response_vehicles"], errors= "coerce" )

# OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 2975-04-21 00:00:00 present at position 141. need errors= "coerce" to make invalid datetimes be NaT

In [ ]:
# connfrming all datatypes match
display(
    ops_18_20.dtypes.equals(ops_21_22.dtypes),
    ops_18_20.dtypes.equals(ops_23_24.dtypes),
    ops_21_22.dtypes.equals(ops_23_24.dtypes),
) # TRUE!

In [ ]:
# check years in each dataframe
for df in all_ops:
    print(df["fiscal_year"].value_counts())

## Combining all datasets 

In [ ]:
# Since all DFs have the same columns, concat all DFs together to 1 big raw DF
ops_18_24 = pd.concat(all_ops, ignore_index = True)

In [ ]:
type(ops_18_24)

In [ ]:
# before of the datatypes for date columns
ops_18_24[date_cols].info()

In [ ]:
# change date time for for all date columns

# for column in date_cols:
#     ops_18_24[column] = pd.to_datetime(ops_18_24[column], unit = "D") # convert to dt
#     ops_18_24[column] = ops_18_24[column].dt.date

In [ ]:
# After the datetime changes to date columns
ops_18_24[date_cols].info

In [ ]:
# ensure all years appear
display(
    ops_18_24["fiscal_year"].value_counts(),
    ops_18_24.shape,
)

## Saving data to GCS as parquet

In [3]:
# saving big DF as parquet to GCS

gcs_path = "gs://calitp-analytics-data/data-analyses/ntd/"
file_name = "sco_operator_data_18_24.parquet"

# ops_18_24.to_parquet(f"{gcs_path}{file_name}")

## Reading in data from GCS as parquet

In [4]:
# read in parquet, ensure it works 
ops_18_24 = pd.read_parquet(f"{gcs_path}{file_name}")



In [6]:
ops_18_24.dtypes

entity_name                                                                                                                      object
fiscal_year                                                                                                                       int64
entity_id                                                                                                                         int64
total_passengers_—_annual_motor_bus                                                                                             float64
total_passengers_—_annual_heavy_rail                                                                                            float64
total_passengers_—_annual_light_rail                                                                                            float64
total_passengers_—_annual_trolley_bus                                                                                           float64
total_passengers_—_annual_ferry_boat            

## Summary findings

In [13]:
# How many unique entities are there 
ops_18_24["entity_name"].nunique()

299

In [17]:
# how many unique entiries are there from each FY
ops_18_24.groupby("fiscal_year")["entity_name"].nunique()

fiscal_year
2018    271
2019    280
2020    281
2021    287
2022    284
2023    278
2024    280
Name: entity_name, dtype: int64

In [10]:
# how many unique entity id are there?
ops_18_24["entity_id"].nunique()

297

In [15]:
# how many have "specialized services" in their name
ops_18_24[ops_18_24["entity_name"].str.contains("Specialized Service")]["entity_name"].nunique()

114

In [20]:
# how many entities do not have "specialized service"
ops_18_24[~ops_18_24["entity_name"].str.contains("Specialized Service")]["entity_name"].nunique()

185

## test aggregations

In [ ]:
# melt big DF so all columns are under 1 column.
melt = pd.melt(
    ops_18_24,
    id_vars= group_list,
    # value_vars = ,
    var_name="metric",
    value_name="metric_unit",
    ignore_index=True,
)

In [ ]:
# inspect melted DF 
display(
    melt.info(),
    melt["fiscal_year"].value_counts(),
    melt["metric"].value_counts(),
    melt["entity_name"].value_counts(),
    melt.head(),
)

In [ ]:
# filter melted df by column categories
melt[melt["metric"].isin(vrh_cols)].info()

In [ ]:
# What does group/agg the melted DF look like?
vrh_total = (
    melt[melt["metric"].isin(vrh_cols)]
    .groupby(["entity_name","entity_id", "fiscal_year","metric"])["metric_unit"]
    .sum()
    .reset_index()
)

vrm_total = (
    melt[melt["metric"].isin(vrm_cols)]
    .groupby(["entity_name","entity_id", "fiscal_year","metric"])["metric_unit"]
    .sum()
    .reset_index()
)

passenger_total =(
    melt[melt["metric"].isin(passenger_cols)]
    .groupby(["entity_name","entity_id", "fiscal_year","metric"])["metric_unit"]
    .sum()
    .reset_index()
)


In [ ]:
# inspect melted DF by just VRH columns in Sacramento
all_totals = [
    vrh_total,
    vrm_total,
    passenger_total
]

for df in all_totals:
    display(df[df["entity_name"].str.contains("Sacramento") & df["metric_unit"]>0].head()) # some operators do not have data for some modes. this makes sense.

In [ ]:
# inspect melted DF by just VRH columns in Sacramento
for df in all_totals:
    display(df[df["entity_name"].str.contains("Albany - Specialized Service") & df["metric_unit"]>0].head())

In [ ]:
operators = melt["entity_name"].unique()

In [ ]:
len(operators)

## read in warehouse data


New syntax to query the warehouse
```
from calitp_data_analysis.sql import get_engine

db_engine = get_engine()

with db_engine.connect() as connection:
    query = ""
    df = pd.read_sql(query, connection)
```

In [3]:
from calitp_data_analysis.sql import get_engine

In [4]:
db_engine = get_engine()

metric_list = [
    "pmt",
    "upt",
    "vrh"
]

In [31]:
# empty list for appending DFs
df_list = []
with db_engine.connect() as connection:
    for metric in metric_list:
        query = f"""
        SELECT
          ntd_id,
          SUM({metric}) AS total_{metric},
          source_agency,
          agency_status,
          year
        FROM
          `cal-itp-data-infra.mart_ntd_funding_and_expenses.fct_service_data_and_operating_expenses_time_series_by_mode_{metric}`
        WHERE
          source_state = "CA"
          AND year >= 2018
        GROUP BY
          ntd_id,
          source_agency,
          agency_status,
          year
        """
        # create df
        metric = pd.read_sql(query,connection) 
        
        # append df to list
        df_list.append(metric)
        

In [32]:
# confirm DFs were populated in list
display(
    len(df_list),
    type(df_list[0]),type(df_list[1]),type(df_list[2]),
    df_list[0].columns, df_list[1].columns, df_list[2].columns
)

3

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

Index(['ntd_id', 'total_pmt', 'source_agency', 'agency_status', 'year'], dtype='object')

Index(['ntd_id', 'total_upt', 'source_agency', 'agency_status', 'year'], dtype='object')

Index(['ntd_id', 'total_vrh', 'source_agency', 'agency_status', 'year'], dtype='object')

In [33]:
# unpack list into separate DFs
ntd_pmt, ntd_upt, ntd_vrh = df_list

In [34]:
ntd_pmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   total_pmt      1291 non-null   float64
 2   source_agency  1614 non-null   object 
 3   agency_status  1614 non-null   object 
 4   year           1614 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 63.2+ KB


In [38]:
display(
    ntd_pmt["year"].sort_values().unique(),
    ntd_upt["year"].sort_values().unique(),
    ntd_vrh["year"].sort_values().unique(),
)

array([2018, 2019, 2020, 2021, 2022, 2023])

array([2018, 2019, 2020, 2021, 2022, 2023])

array([2018, 2019, 2020, 2021, 2022, 2023])

In [52]:
merge_1 = ntd_pmt.merge(ntd_upt, on=["ntd_id", "year","source_agency","agency_status"], how = "inner")

In [53]:
merge_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 1613
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   total_pmt      1291 non-null   float64
 2   source_agency  1614 non-null   object 
 3   agency_status  1614 non-null   object 
 4   year           1614 non-null   int64  
 5   total_upt      1291 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 88.3+ KB


In [54]:
ntd_all_metrics = merge_1.merge(ntd_vrh, on=["ntd_id", "year","source_agency","agency_status"], how = "inner")

In [55]:
ntd_all_metrics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1614 entries, 0 to 1613
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ntd_id         1524 non-null   object 
 1   total_pmt      1291 non-null   float64
 2   source_agency  1614 non-null   object 
 3   agency_status  1614 non-null   object 
 4   year           1614 non-null   int64  
 5   total_upt      1291 non-null   float64
 6   total_vrh      1291 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 100.9+ KB


In [58]:
col_order =[
    "ntd_id",
    "source_agency",
    "agency_status",
    "year",
    "total_upt",
    "total_vrh",
    "total_pmt"
]

ntd_all_metrics = ntd_all_metrics[col_order]

In [59]:
ntd_all_metrics.head(10)

,ntd_id,source_agency,agency_status,year,total_upt,total_vrh,total_pmt
0,90198,City of Porterville (COLT) - Transit Department,Active,2022,260879.0,41460.0,0.0
1,90198,City of Porterville (COLT) - Transit Department,Active,2020,522056.0,47356.0,0.0
2,90198,City of Porterville (COLT) - Transit Department,Active,2023,NaN,NaN,NaN
3,90198,City of Porterville (COLT) - Transit Department,Active,2019,635559.0,52834.0,0.0
4,90198,City of Porterville (COLT) - Transit Department,Active,2021,174595.0,30773.0,0.0
5,90198,City of Porterville (COLT) - Transit Department,Active,2018,648649.0,52799.0,0.0
6,90036,Orange County Transportation Authority (OCTA),Active,2020,33009047.0,2232009.0,155734272.0
7,90036,Orange County Transportation Authority (OCTA),Active,2023,32674688.0,2076903.0,137085597.0
8,90036,Orange County Transportation Authority (OCTA),Active,2022,27753507.0,1897023.0,132745034.0
9,90036,Orange County Transportation Authority (OCTA),Active,2018,42201857.0,2590593.0,214680839.0
